In [ ]:
from transformers import pipeline

In [ ]:
nlp = pipeline("question-answering")

In [13]:
OS_original_text = ''
with open("OS_extended.txt") as file:
    OS_original_text = file.read()
print(OS_original_text)

print(nlp(question="What are the different OS for personal computers?", context=OS_original_text))
print(nlp(question="What does operating system coordinates?", context=OS_original_text))
print(nlp(question="What is necessary for a programs to run?", context=OS_original_text))

An operating system is the most important software that runs on a computer. It manages the computer's memory and processes, as well as all of its software and hardware. It also allows you to communicate with the computer without knowing how to speak the computer's language. Without an operating system, a computer is useless.
Your computer's operating system (OS) manages all of the software and hardware on the computer. Most of the time, there are several different computer programs running at the same time, and they all need to access your computer's central processing unit (CPU), memory, and storage. The operating system coordinates all of this to make sure each program gets what it needs.
Operating systems usually come pre-loaded on any computer you buy. Most people use the operating system that comes with their computer, but it's possible to upgrade or even change operating systems. The three most common operating systems for personal computers are Microsoft Windows, macOS, and Linu

Using Model and tokenizer

In [14]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

In [16]:
!pip install transformers sentencepiece

Defaulting to user installation because normal site-packages is not writeable


In [18]:
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad", return_dict=False)

In [20]:
questions = [
    "What are the different OS for personal computers?",
    "What does operating system coordinates?",
    "What is necessary for a programs to run?",
]

for question in questions:
    inputs = tokenizer.encode_plus(question, OS_original_text, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    answer_start_scores, answer_end_scores = model(**inputs)

    answer_start = torch.argmax(
        answer_start_scores
    )  # Get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    print(f"Question: {question}")
    print(f"Answer: {answer}\n")

Question: What are the different OS for personal computers?
Answer: microsoft windows, macos, and linux

Question: What does operating system coordinates?
Answer: to make sure each program gets what it needs

Question: What is necessary for a programs to run?
Answer: access your computer's central processing unit ( cpu ), memory, and storage

